# Zadanie
### Autor: Michał Karp
Bracie, wytrenuj sobie segmentację na dostarczonym datasetcie.

# Ograniczenia

- GPU Colab T4
- max 10 min na gpu (bez pobierania danych)
- rozwiązanie będzie testowanie w środowisku bez dostępu do internetu
- nie możesz trenować na zbiorze walidacyjnym ale możesz go użyć do early stopping i innych tego typu technik

# Ocenianie
Metryką jest Dice coefficient. Wzór na Dice coefficient to:

$$
\text{Dice} = \frac{2 \cdot TP}{2 \cdot TP + FP + FN}
$$

Gdzie:

- TP – True Positives (prawidłowo przewidziane piksele obiektu)

- FP – False Positives (piksele tła przewidziane jako obiekt)

- FN – False Negatives (piksele obiektu przewidziane jako tło).

Jeśli Dice cooficent jest poniżej 0.7, otrzymasz 0 punktów. Jeśli powyżej 0.98, otrzymasz 100 punktów. Pomiędzy tymi wartościami wynik rośnie liniowo.

Inne popularne metryki to F1 score, accuracy itp, ale my posłużymy się tą.

# Kod startowy

In [ ]:
##### NIE ZMIENIAJ TEJ KOMÓRKI #####

import kagglehub
import os
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import torchvision

In [ ]:
##### NIE ZMIENIAJ TEJ KOMÓRKI #####

path = kagglehub.dataset_download("hngngn/portrait-segmentation-128x128")
print("Path to dataset files:", path)

In [ ]:
##### NIE ZMIENIAJ TEJ KOMÓRKI (no ok możesz dostosować batch size) #####

BASE_DIR = Path(path)

TRAIN_IMG_DIR = BASE_DIR / "xtrain"
TRAIN_MSK_DIR = BASE_DIR / "ytrain"

VAL_IMG_DIR = BASE_DIR / "xtest"
VAL_MSK_DIR = BASE_DIR / "ytest"

to_tensor = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

mask_to_tensor = transforms.Compose([
    transforms.Resize((128, 128), interpolation=transforms.InterpolationMode.NEAREST_EXACT),
    transforms.ToTensor(),
])

class PortraitSegDataset(Dataset):
    def __init__(self, img_dir, msk_dir, img_transform=None, msk_transform=None):
        self.img_dir = Path(img_dir)
        self.msk_dir = Path(msk_dir)
        self.img_transform = img_transform
        self.msk_transform = msk_transform

        self.names = sorted({p.name for p in self.img_dir.iterdir() if p.suffix.lower() == ".png"})

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        name = self.names[idx]
        img = Image.open(self.img_dir / name).convert("RGB")
        msk = Image.open(self.msk_dir / name).convert("L")

        if self.img_transform:
            img = self.img_transform(img)

        if self.msk_transform:
            msk = self.msk_transform(msk)

        return img, msk

train_ds = PortraitSegDataset(TRAIN_IMG_DIR, TRAIN_MSK_DIR, img_transform=to_tensor, msk_transform=mask_to_tensor)
val_ds = PortraitSegDataset(VAL_IMG_DIR, VAL_MSK_DIR, img_transform=to_tensor, msk_transform=mask_to_tensor)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, drop_last=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

print(f"Train set: {len(train_ds)} przykładów  •  Val set: {len(val_ds)} przykładów")

fig, axes = plt.subplots(1, 5, figsize=(15, 3))

for i in range(5):
    img, msk = train_ds[i]
    ax = axes[i]
    ax.imshow(img.permute(1, 2, 0))
    ax.imshow(msk.squeeze(), alpha=0.4)
    ax.axis("off")

plt.tight_layout()
plt.show()

# Rozwiązanie
W tej sekcji wytrenuj model i umieść rozwiązanie.

In [ ]:
# W tej sekcji wytrenuj swój model

model = ...

In [ ]:
def segment_image(valid_dataset: list):
  return [torch.zeros((1, 128, 128), dtype=torch.bool) for _ in range(len(valid_dataset))]

# Wynik

In [ ]:
def dice_coefficient(targets, predictions):
    dice_scores = []
    for target, pred in zip(targets, predictions):
        intersection = (target & pred).sum().float()
        union = target.sum().float() + pred.sum().float()
        dice = (2. * intersection) / (union + 1e-8)
        dice_scores.append(dice)

    return torch.mean(torch.stack(dice_scores)).item()

In [ ]:
def scale_dice(dice):
    if dice < 0.7:
        return 0.0
    elif dice > 0.98:
        return 1.0
    else:
        return 100 * (dice - 0.7) / (0.98 - 0.7)

In [ ]:
result = dice_coefficient(
    list(map(lambda X: X[1].bool(), val_ds)),
    segment_image(list(map(lambda X: X[0], val_ds)))
)

print(f"Dice cooficent:", result)
print(f"Score:", scale_dice(result))